In [12]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 22 20:56:04 2023

@author: zhangzhao
"""

import random
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils import data
import train_fun as tf
import data_prepare as dpp
import time
import preprocessing as pp

##################################数据载入#####################################################
all_lab = pd.read_csv(r'D:\20230816\数据汇总\新LTB\ML\all_lab.csv', header=None).values.astype(np.float32)
all_feat = pd.read_csv(r'D:\20230816\数据汇总\新LTB\ML\all_feat.csv', header=None).values.astype(np.float32)
all_wave = pd.read_table(r'D:\20230816\数据汇总\新LTB\ML\wl.csv', header=None, sep=';', usecols=[0]).values.astype(np.float32)


cort_index = all_lab[:, 0].argsort()#按顺序排列
all_lab = all_lab[cort_index, :]
all_feat = all_feat[cort_index, :]
all_feat_normal = np.zeros(np.shape(all_feat))
print('数据读取完成')
####################################数据归一化##############################################
for i in range(len(all_feat[:,0])):
    all_feat_normal[i] = pp.Normalization(all_feat[i],method='total_intensity')
print('归一化完成')
###################################进行随机采样#################################################
avg_num = 0
avg_step = 20
all_model_performance = []
num_sample = 7
num_spec = len(all_feat) // num_sample
all_index = list(np.arange(len(all_feat)))

for i in range(2):
    avg_num = avg_num + avg_step
    print('%d张平均开始' % avg_num)
    index_random = []
    for j in range(num_sample):
        index = random.sample(all_index[j * num_spec:(j + 1) * num_spec], avg_num * 10)
        index = np.array(index)
        index_random.append(index)
    index_random = np.array(index_random)
    index_random = index_random.ravel()
    random_feat = all_feat_normal[index_random, :]
    random_lab = all_lab[index_random, :]
    random_lab = random_lab.reshape(-1, 1)
    ###############################光谱平均处理及特征选择#####################################################
    avg_feat = dpp.spec_avg_normal(random_feat, avg_num)
    avg_lab = dpp.spec_avg_normal(random_lab, avg_num)
    scores = dpp.score_weighted(avg_feat, avg_lab)
    num_feat = 150
    index_feat = dpp.select_best(scores, num_feat)
    select_feat = avg_feat[:, index_feat]
    pd.DataFrame(all_wave[index_feat, :]).to_csv('20230829/select_wave_' + str(avg_num) + '.csv', header=False, index=False)
    pd.DataFrame(all_feat[1399, index_feat]).to_csv('20230829/select_feat_' + str(avg_num) + '.csv', header=False, index=False)
    pd.DataFrame(all_feat[1399, :]).to_csv('20230829/feat.csv', header=False, index=False)
    pd.DataFrame(scores).to_csv('20230829/scores_' + str(avg_num) + '.csv', header=False, index=False)


    ##############################数据集划分及包装##########################################################
    batch_size = 64
    index_test = [1, 6]
    train_feat, train_lab, test_feat, test_lab = tf.stand_divid(select_feat, avg_lab, num_sample, index_test)
    train_feat = tf.a2t(train_feat)
    train_lab_tensor = tf.a2t(train_lab)
    test_feat = tf.a2t(test_feat)
    test_lab_tensor = tf.a2t(test_lab)
    train_data = tf.load_array((train_feat, train_lab_tensor), batch_size)
    ###############################BPNN模型#################################################
    class BPNN(torch.nn.Module):
        def __init__(self):
            super(BPNN, self).__init__()
            self.fc1 = nn.Linear(200, 100)
            self.fc2 = nn.Linear(100, 1)

        def forward(self, x):
            x = f.tanh(self.fc1(x))
            y = self.fc2(x)
            return y
    train_lab_tensor.numpy()
    if torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'
    logdir = os.getcwd()
    print('模型训练开始')
    model = tf.BPNN()
    x,y = tf.small_model_train(model, train_data, train_feat, train_lab_tensor, test_feat, test_lab_tensor, device, logdir)
    print('模型训练结束')
    #############################模型性能评估################################################################
    train_pred_lab = model(train_feat).detach().numpy()
    test_pred_lab = model(test_feat).detach().numpy()
    r2 = dpp.compute_r2(train_lab, train_pred_lab)
    slope = dpp.compute_slope(train_lab, train_pred_lab)
    rec = dpp.compute_MRE(train_lab, train_pred_lab)
    rep = dpp.compute_MRE(test_lab, test_pred_lab)
    rsd = dpp.compute_rsd(test_pred_lab, 2)
    model_performance = np.array([r2, slope, rec, rep, rsd])
    pd.DataFrame(model_performance).to_csv('20230829/model_performance_' + str(avg_num) + '.csv', index=False, header=False)
    print('模型性能评估完成')
    all_model_performance.append(model_performance)
    ############################获取均值与标准差#####################################################################
    sd_mean = dpp.get_sd_mean(train_lab / 50, train_pred_lab / 50, test_lab / 50, test_pred_lab / 50, 9)
    pd.DataFrame(sd_mean).to_csv('sd_mean_' + str(avg_num) + '.csv', index=False, header=False)

pd.DataFrame(all_model_performance).to_csv('20230829/all_model_performance_' + str(avg_num) + '.csv', index=False, header=False)

数据读取完成
归一化完成
20张平均开始
(20,)
模型训练开始


epoch: [166/5000]: 100%|██████████| 1/1 [00:00<00:00, 125.33it/s]


KeyboardInterrupt: 

In [ ]:
'''
##################################光谱平均处理#####################################################
avg_step = 100
# all_feat=dpp.spec_avg_normal(all_feat,avg_step) #####平均光谱的时候使用
##################################光谱归一化及特征选择#####################################################
all_feat_normalspec = dpp.normal_spec(all_feat)
scores = dpp.score(all_feat_normalspec, all_lab)
num_feat = 150
index_feat = dpp.select_best(scores, num_feat)
select_feat = all_feat_normalspec[:, index_feat]
print('光谱归一化及特征选择完成')

##################################数据集划分及包装#####################################################
batch_size = 4000
index_test = []
train_feat, train_lab, test_feat, test_lab = tf.stand_divid(select_feat, all_lab, 9, index_test)
train_feat = tf.a2t(train_feat)
train_lab_tensor = tf.a2t(train_lab)
test_feat = tf.a2t(test_feat)
test_lab_tensor = tf.a2t(test_lab)
train_data = tf.load_array((train_feat, train_lab_tensor), batch_size)
##################################模型训练#####################################################
model = tf.bpnn()
print('测试标签：%f的模型训练开始' % (test_lab[0]))
model = tf.model_train(model, train_data, train_feat, train_lab, test_feat, test_lab)
torch.save(model, 'model_' + str(test_lab[0]) + '.pth')
print('测试标签：%f的模型训练完成' % (test_lab[0]))
##################################模型性能评估#####################################################
'''
